In [1]:
!pip install openai-clip

In [2]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from torch import nn
from torchvision.transforms import Compose, Resize, ToTensor
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
import torch.optim as optim
from tqdm.autonotebook import tqdm
from collections import OrderedDict
import torch.nn.functional as F
from torch.optim import lr_scheduler
# import netron
import torch.onnx
from torch.autograd import Variable

_tokenizer = _Tokenizer()

/var/folders/pn/q7mmpvpx5n36lk_j6p9gqk4c0000gn/T/ipykernel_88318/4148449913.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Load the model
device = torch.device('cpu')
model, preprocess = clip.load('ViT-B/32', device)
device
# model.dtype

device(type='cpu')

In [4]:
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection
        self.dtype = clip_model.dtype

    def forward(self, prompts, tokenized_prompts):
        x = prompts + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

        return x


class PromptLearner(nn.Module):
    def __init__(self,  classnames, clip_model):
        super().__init__()
        n_cls = len(classnames)
        n_ctx = 4
        ctx_init = 'a photo_of a'
        dtype = clip_model.dtype
        ctx_dim = clip_model.ln_final.weight.shape[0]
        vis_dim = clip_model.visual.output_dim
        clip_imsize = clip_model.visual.input_resolution
        # cfg_imsize = cfg.INPUT.SIZE[0]
        # assert cfg_imsize == clip_imsize, f"cfg_imsize ({cfg_imsize}) must equal to clip_imsize ({clip_imsize})"

        if ctx_init:
            # use given words to initialize context vectors
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init).to(device)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt).type(dtype)
            ctx_vectors = embedding[0, 1 : 1 + n_ctx, :].to(device)
            prompt_prefix = ctx_init
        else:
            # random initialization
            ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=dtype)
            nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)

        print(f'Initial context: "{prompt_prefix}"')
        print(f"Number of context words (tokens): {n_ctx}")

        self.ctx = nn.Parameter(ctx_vectors).to(device)

        self.meta_net = nn.Sequential(OrderedDict([
            ("linear1", nn.Linear(vis_dim, vis_dim // 16)),
            ("relu", nn.ReLU(inplace=True)),
            ("linear2", nn.Linear(vis_dim // 16, ctx_dim))
        ])).to(device)
        
        # if cfg.TRAINER.COCOOP.PREC == "fp16":
        # self.meta_net.half()

        classnames = [name.replace("_", " ") for name in classnames]
        name_lens = [len(_tokenizer.encode(name)) for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]

        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts]).to(device)  # (n_cls, n_tkn)
        with torch.no_grad():
            embedding = clip_model.token_embedding(tokenized_prompts).to(device).type(dtype)

        # These token vectors will be saved when in save_model(),
        # but they should be ignored in load_model() as we want to use
        # those computed using the current class names
        self.register_buffer("token_prefix", embedding[:, :1, :])  # SOS
        self.register_buffer("token_suffix", embedding[:, 1 + n_ctx :, :])  # CLS, EOS

        self.n_cls = n_cls
        self.n_ctx = n_ctx
        self.tokenized_prompts = tokenized_prompts  # torch.Tensor
        self.name_lens = name_lens
    
    def construct_prompts(self, ctx, prefix, suffix, label=None):
        # dim0 is either batch_size (during training) or n_cls (during testing)
        # ctx: context tokens, with shape of (dim0, n_ctx, ctx_dim)
        # prefix: the sos token, with shape of (n_cls, 1, ctx_dim)
        # suffix: remaining tokens, with shape of (n_cls, *, ctx_dim)

        if label is not None:
            prefix = prefix[label]
            suffix = suffix[label]

        prompts = torch.cat(
            [
                prefix,  # (dim0, 1, dim)
                ctx,     # (dim0, n_ctx, dim)
                suffix,  # (dim0, *, dim)
            ],
            dim=1,
        ).to(device)

        return prompts

    def forward(self, im_features):
        prefix = self.token_prefix.to(device)
        suffix = self.token_suffix.to(device)

        ctx = self.ctx                     # (n_ctx, ctx_dim)
        print(f'im_features:{im_features.shape}')
        print(f'self.meta_net:{self.meta_net}')
        bias = self.meta_net(im_features)  # (batch, ctx_dim)

        bias = bias.unsqueeze(1)           # (batch, 1, ctx_dim)
        ctx = ctx.unsqueeze(0)             # (1, n_ctx, ctx_dim)

        ctx_shifted = ctx + bias           # (batch, n_ctx, ctx_dim)
        
        prompts = []
        for ctx_shifted_i in ctx_shifted:
            ctx_i = ctx_shifted_i.unsqueeze(0).expand(self.n_cls, -1, -1)
            pts_i = self.construct_prompts(ctx_i, prefix, suffix)  # (n_cls, n_tkn, ctx_dim)
            prompts.append(pts_i)
        prompts = torch.stack(prompts)
        
        return prompts


class CustomCLIP(nn.Module):
    def __init__(self, classnames, clip_model):
        super().__init__()
        self.prompt_learner = PromptLearner( classnames, clip_model).to(device)
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts.to(device)
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model).to(device)
        self.logit_scale = clip_model.logit_scale
        self.dtype = clip_model.dtype

    def forward(self, image, label=None):
        tokenized_prompts = self.tokenized_prompts
        logit_scale = self.logit_scale.exp()

        image_features = self.image_encoder(image.type(self.dtype)).type(self.dtype).to(device)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        print(f'image:{image.shape}')
        print(f'image_features:{image_features.shape}')
        prompts = self.prompt_learner(image_features)# 这里有不同，之前的prompt直接就是一个prompt all 的vec了
        # (batch_size ,n_cls, n_tkn, ctx_dim)
        # print(0.1)
        logits = []
        for pts_i, imf_i in zip(prompts, image_features):
            text_features = self.text_encoder(pts_i.type(self.dtype), tokenized_prompts.type(self.dtype)).type(self.dtype).to(device)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            l_i = logit_scale * imf_i @ text_features.t()
            logits.append(l_i)
        logits = torch.stack(logits)
        print(f'self.prompt_learner.training:{self.prompt_learner.training}')
        print(f'logits:{logits}')
        return logits

In [5]:
customCLIP = CustomCLIP(['1','2'], model).to(device)

Initial context: "a photo of a"
Number of context words (tokens): 4


In [9]:
from torchviz import make_dot

batch_size, dim = 16, 512
image_input = torch.randn(batch_size, dim).to(device)
output = customCLIP.prompt_learner(image_input)
# Get parameters with requires_grad=True
params = {name: param for name, param in customCLIP.prompt_learner.named_parameters() if param.requires_grad}

# Visualize the model
make_dot(output, params=params).render("prompt_learner", format="png")

im_features:torch.Size([16, 512])
self.meta_net:Sequential(
  (linear1): Linear(in_features=512, out_features=32, bias=True)
  (relu): ReLU(inplace=True)
  (linear2): Linear(in_features=32, out_features=512, bias=True)
)


'prompt_learner.png'

In [11]:
pts_i = torch.randn([10, 77, 512])
imf_i = torch.randn([512])
output = customCLIP.text_encoder(pts_i,imf_i)
params = {name: param for name, param in customCLIP.text_encoder.named_parameters() if param.requires_grad}
make_dot(output, params=params).render("text_encoder", format="png")

'text_encoder.png'